# Entrada al algoritmo

- object_list = [(1,2), (2,2), (1,1), (3,2)]
- frequencies_list = [5, 10, 10, 15]
- width = 20

# Preprocesamiento

- rectangles_list = [Rectangle(p[0],p[1]) for p in object_list]

# Ejemplo de salida

- positions_list = [(0,0),(3,0),...]
- order_list = [3,2,1,3,3,2,1,1,1,1,2]

# Imports

In [35]:
%load_ext memory_profiler

# %matplotlib notebook
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.patches as patches
import numpy as np

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Recursive Packing

Esta función se encarga de encasillar rectangulos en espacios acotados.

Parámetros de entrada:
- **S2**: Rectángulo acotado
- **rectangles_list**: Lista de *Rectangles* a distribuir ordenados por área decreciente

Funciones auxiliares:
- **fits**: Comprueba si algún rectángulo cabe en *S2*. Devuelve su posición en tal caso y *None* en otro.

pack(S2,) -> coloca el rectangulo pasado por entrada en el espacio acotado S. Devuelve el espacio no usado para colocar el rectangulo S, este espacio seran dos subespacios acotados S3 y S4


In [36]:
class Rectangle:
    def __init__(self, width, height, x=None, y=None):
        self.width = width
        self.height = height
        self.area = width*height
        self.x = x
        self.y = y
        
    def __lt__(self, other):
        return self.area < other.area
    
    def __str__(self):
        return "Coordinates: ({}, {})\tShape: ({}, {})\t Area:{}".format(self.x, self.y, self.width, self.height, self.area)

In [37]:
def recursive_packing(S2):
    valid_rectangle_position = fits(S2)
    
    if valid_rectangle_position == None:
        return
    else:
        S3, S4 = pack(S2, valid_rectangle_position)
        
        if S3.area > S4.area:
            recursive_packing(S3)
            
            if(S4.area > 0):
                recursive_packing(S4)
        else:
            recursive_packing(S4)
            
            if(S3.area > 0):
                recursive_packing(S3)
        

In [38]:
def fits(s):
    for i, r in enumerate(rectangles_list):
        if r.width <= s.width and r.height <= s.height and frequencies_list[i] > 0:
            return i

In [39]:
def pack(S2, valid_rectangle_position):
    R = rectangles_list[valid_rectangle_position]
    
    S3 = Rectangle(        R.width     , S2.height - R.height,      S2.x      , S2.y + R.height )
    S4 = Rectangle( S2.width - R.width ,       S2.height     , S2.x + R.width ,      S2.y       )
    
    output.append(Rectangle(R.width, R.height, S2.x, S2.y))
    frequencies_list[valid_rectangle_position] -= 1
        
    return S3, S4

# Ejemplo Recursive Packing

In [40]:
# width = 10
# height = 15

# object_list = [(1,2), (2,2), (1,1), (3,2)]
# frequencies_list = [5, 10, 10, 15]

# output = []
# rectangles_list = [Rectangle(p[0],p[1]) for p in object_list]
# rectangles_list = sorted(rectangles_list, reverse=True)



# Strip = Rectangle(width, height, 0, 0)

# # print(Strip)

# # print("Rectangles list:")
# # for r in rectangles_list:
# #     print(r)
    
# # print("Rectangles list sorted by area:")
# # for r in sorted(rectangles_list, reverse=True):
# #     print(r)
    
# recursive_packing(Strip)

# # for r in output:
# #     print(r)

In [41]:
# %matplotlib notebook
# import matplotlib.pyplot as plt
# import matplotlib.animation
# import matplotlib.patches as patches
# import numpy as np

# def animate(i):
#     x = output[i].x
#     y = output[i].y
#     w = output[i].width
#     h = output[i].height

#     r = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='b', facecolor='r', fill=True)

#     ax.add_patch(r)
#     rx, ry = r.get_xy()
#     cx = rx + r.get_width()/2.0
#     cy = ry + r.get_height()/2.0
#     ax.annotate('A', (cx, cy), color='w', weight='bold', fontsize=6, ha='center', va='center')

# def plot():
#     fig, ax = plt.subplots(figsize=(5,5))
#     ax.set_xlim([0, width])
#     ax.set_ylim([0, height])

#     ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(output), interval=200)

# Packing

In [42]:
def pack_unbounded(S, rectangle_to_pack_position):
    R = rectangles_list[rectangle_to_pack_position]
    
    S1 = Rectangle(        S.width    ,   np.inf  ,      S.x      , S.y + R.height )
    S2 = Rectangle( S.width - R.width , R.height , S.x + R.width ,      S.y       )
    
    output.append(Rectangle(R.width, R.height, S.x, S.y))
    frequencies_list[rectangle_to_pack_position] -= 1
        
    return S1, S2

In [43]:
def first_non_zero(l):
    for i, item in enumerate(l):
        if(item > 0):
            return i

In [44]:
def packing(S):
    height = 0
    
    while np.any(frequencies_list):
        pos = first_non_zero(frequencies_list)
        S1, S2 = pack_unbounded(S, pos)
        height += rectangles_list[pos].height
        
        S = S1
        
        recursive_packing(S2)
        
    return height

# Ejemplo packing

In [12]:
width = 10
height = 15

object_list = [(1,2), (2,2), (1,1), (3,2)]
frequencies_list = [5, 10, 10, 15]

output = []
rectangles_list = [Rectangle(p[0], p[1]) for p in object_list]
rectangles_list = sorted(rectangles_list, reverse=True)



Strip = Rectangle(width, np.inf, 0, 0)

    
packing(Strip)

# for r in output:
#     print(r)

In [13]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_xlim([0, width])
ax.set_ylim([0, height])

def animate(i):
    x = output[i].x
    y = output[i].y
    w = output[i].width
    h = output[i].height

    r = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='b', facecolor='r', fill=True)

    ax.add_patch(r)
    rx, ry = r.get_xy()
    cx = rx + r.get_width()/2.0
    cy = ry + r.get_height()/2.0
    ax.annotate('A', (cx, cy), color='w', weight='bold', fontsize=6, ha='center', va='center')

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(output), interval=200)

# Test con dataset de prueba

In [14]:
%load_ext memory_profiler

with open('data\\data\\data-changed\\CX\\50.txt') as f:
    line = f.readline().split(' ')
    width = int(line[0])
    
    n = int(f.readline().split(' ')[0])
    
    print(width)
    print(n)
    
    rectangles_list = []
    frequencies_list = []
    output = []
    
    for i in range(n):
        line = f.readline().split('\t')
        rectangles_list.append(Rectangle(int(line[0]), int(line[1])))
        frequencies_list.append(100)
        
        Strip = Rectangle(width, np.inf, 0, 0)
        
    rectangles_list = sorted(rectangles_list, reverse=True)
        
    %time final_height = packing(Strip)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
400
50
Wall time: 44.9 ms


In [15]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_xlim([0, width])
ax.set_ylim([0, final_height])

def animate(i):
    x = output[i].x
    y = output[i].y
    w = output[i].width
    h = output[i].height

    r = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='b', facecolor='r', fill=True)

    ax.add_patch(r)
    rx, ry = r.get_xy()
    cx = rx + r.get_width()/2.0
    cy = ry + r.get_height()/2.0
#     ax.annotate('A', (cx, cy), color='w', weight='bold', fontsize=6, ha='center', va='center')

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(output), interval=10)

# Ejemplo inventado

In [1]:
%load_ext memory_profiler

width = 50

n = 20

rectangles_list = []
frequencies_list = []
output = []

for i in range(n):
    w = np.random.randint(1,16)
    h = np.random.randint(1,16)
    rectangles_list.append(Rectangle(w, h))
    frequencies_list.append(np.random.randint(100000))

    Strip = Rectangle(width, np.inf, 0, 0)
    
# for r in rectangles_list:
#     print(r)
    
print(frequencies_list)

rectangles_list = sorted(rectangles_list, reverse=True)

%time final_height = packing(Strip)

NameError: name 'np' is not defined

In [17]:
fig, ax = plt.subplots(figsize=(5,5))
ax.set_xlim([0, width])
ax.set_ylim([0, 1000])

def animate(i):
    x = output[i].x
    y = output[i].y
    w = output[i].width
    h = output[i].height

    r = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='b', facecolor='r', fill=True)

    ax.add_patch(r)
    rx, ry = r.get_xy()
    cx = rx + r.get_width()/2.0
    cy = ry + r.get_height()/2.0
#     ax.annotate('A', (cx, cy), color='w', weight='bold', fontsize=6, ha='center', va='center')

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(output), interval=10)

In [18]:
# fig, ax = plt.subplots(figsize=(5,5))
# ax.set_xlim([0, width])
# ax.set_ylim([0, final_height])

# for i, r in enumerate(output):
#     x = output[i].x
#     y = output[i].y
#     w = output[i].width
#     h = output[i].height

#     r = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='b', facecolor='r', fill=True)
#     ax.add_patch(r)
    
# plt.show()

# Otro ejemplo

In [33]:
def total_waste(rectangles_list, total_width, total_height):
    filled_area = 0
    
    for r in rectangles_list:
        filled_area += r.area
        
    return total_width*total_height - filled_area
            

In [53]:
filled_area = 0
final_height = 0
    
with open('data\\data\\data\\2004.Burke.n\\n\\n13.txt') as f:
    line = f.readline().split(' ')
    width = int(line[0])
    
    n = int(f.readline().split(' ')[0])
    
    rectangles_list = []
    frequencies_list = []
    output = []
    
    for i in range(n):
        line = f.readline().split(' ')
        rectangles_list.append(Rectangle(int(line[0]), int(line[1])))
        frequencies_list.append(1)
        
        Strip = Rectangle(width, np.inf, 0, 0)
        
    rectangles_list = sorted(rectangles_list, reverse=True)
        
    %time final_height = packing(Strip)
    
    print("Total area: {}".format(width*final_height))
    print("Total waste: {}".format(total_waste(output, width, final_height)))
    print("Percentage wasted: {}".format(total_waste(output, width, final_height)/(width*final_height)*100))

Wall time: 1.03 s
Total area: 625280
Total waste: 10880
Percentage wasted: 1.7400204708290685


In [ ]:
filled_area = 0
final_height = 0
    
with open('data\\data\\data\\2004.Burke.n\\n\\nlist.txt') as f:
    line = f.readline().split(' ')
    width = int(line[0])
    
    n = int(f.readline().split(' ')[0])
    
    rectangles_list = []
    frequencies_list = []
    output = []
    
    for i in range(n):
        line = f.readline().split(' ')
        rectangles_list.append(Rectangle(int(line[0]), int(line[1])))
        frequencies_list.append(1)
        
        Strip = Rectangle(width, np.inf, 0, 0)
        
    rectangles_list = sorted(rectangles_list, reverse=True)
        
    %time final_height = packing(Strip)
    
    print("Total area: {}".format(width*final_height))
    print("Total waste: {}".format(total_waste(output, width, final_height)))
    print("Percentage wasted: {}".format(total_waste(output, width, final_height)/(width*final_height)*100))